In [65]:
import time
t=time.time()
import random

blockzone=[]
for i in range(4):
    blockzone.append((random.randint(0,3)==2)*random.randint(0,4))
blockzone

zone=[]
for i in range(16):
    zone.append(random.randint(0,4)+(i>12)*(random.randint(0,5)))

sat =[[1,10,],[1,4],[7,10],[7,4],[0,9],[0,3],[0,6],[9,3],[3,7],[6,9]
     ,[0,10],[0,7],[9,7],[9,4],[6,4],[6,1],[3,1],[3,10],[12,1],[13,10]
     ,[14,11],[14,7],[15,0],[15,4]]

for i in range(5,10):
    sat.append([12,i])
for i in range(2,7):
    sat.append([13,i])
for i in range(0,4):
    sat.append([14,i])
for i in range(8,12):
    sat.append([15,i])

contflow=[]
for i in range(16):
    contflow.append(5+(i>=12)*5)
changeflow=[]
for i in range(16):
    changeflow.append(2+(i>=12)*3)
    
weights=[]
for i in range(16):
    weights.append(1)
for i in range(12,16):
    weights[i]/=3
    
times=[]
for i in range(16):
    times.append(random.randint(0,60))

timeweights=[]
for i in range(16):
    timeweights.append(1)
    
for i in range(12):
    timeweights[i]/=2

print(weights)
print(contflow)
print(changeflow)
print()
print(times)
print(timeweights)
print(time.time()-t)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5]

[43, 25, 39, 11, 4, 26, 57, 35, 37, 14, 59, 39, 37, 21, 10, 19]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1]
0.002994537353515625


In [66]:
def possiblemask(mask):
    for i in sat:
        if not ((mask[i[0]]==0) or (mask[i[1]]==0)):
            return 0
    return 1

In [67]:
def genmask():
    a=0
    while(not a): 
        mask=[]
        for i in range(16):
            mask.append(random.randint(0,1))
        a=possiblemask(mask)
    return mask
prevmask=genmask()

In [68]:
def tomask(a):
    i=0
    mask=[]
    for i in range(16):
        mask.append(a%2)
        a//=2
    return mask

In [69]:
def greed(mask,zone):
    ans=0
    for i in range(16):
        if(prevmask[i]==mask[i]):
            ans+=max(zone[i],contflow[i])*mask[i]*weights[i]*(((times[i]*timeweights[i])/30)**2)
        else:
            ans+=max(zone[i],changeflow[i])*mask[i]*weights[i]*(((times[i]*timeweights[i])/30)**2)
    return ans

In [70]:
def noblock(mask,blockzone):
    blockmask=[]
    for i in range(12):
        blockmask.append(0)
    for j in range(0,4):
        if(blockzone[j]):
            for i in range(j*3,3+j*3):
                blockmask[i]=1
            for i in range(j*3+3,j*3+11,4):
                blockmask[i%12]=1
    for i in range(12):
        if(mask[i] and blockmask[i]):
            return 0
    return 1

In [71]:
pt=time.time()

def greedchoose(masksize, zone):
    maxv=0
    maxmask=[]
    for i in range(1 << masksize):
        mask = tomask(i+1)
        a = greed(mask, zone) * possiblemask(mask)*noblock(mask,blockzone)
        if(maxv < a):
            maxv = a
            maxmask = mask
    return [maxmask, maxv]

print(times)
print(timeweights)
print()
print(zone)
print(weights)
print()
print(blockzone)
print(prevmask)
print(greedchoose(16, zone))
print(time.time()-t)
print(time.time()-pt)

[43, 25, 39, 11, 4, 26, 57, 35, 37, 14, 59, 39, 37, 21, 10, 19]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1]

[1, 1, 4, 2, 3, 4, 2, 4, 4, 4, 3, 2, 0, 3, 4, 8]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

[1, 0, 0, 2]
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0], 4.830462962962963]
3.1727097034454346
2.436627149581909


In [72]:
mask=[]
for i in range(16):
    mask.append(0)
noblock(mask,blockzone)

1